# imports

In [23]:
import pandas as pd
import os
import numpy as np
from scipy.stats import lognorm, nbinom, gamma
from datetime import timedelta
import psycopg2
from collections import deque

In [2]:
conn_params = {
    "host" : "srvdados",
    "database" : "postgres",
    "user":"compras",
    "password": "pecist@compr@s2024"
}

In [3]:
ve = """
    SELECT 
    pp.cod_pro,
    pp.cd_loja,
    pp.dt_emissao::date AS dt_emissao,
    SUM(pp.qtde_ven) AS vendas
FROM "D-1".prod_ped pp
JOIN "D-1".cliente c 
    ON pp.codcli = c.codcli
WHERE 
    pp.tipped = 'V'
    AND pp.dt_emissao >= DATE '2019-01-01'
    AND pp.cd_loja NOT IN ('01','08','09')
    and c.codcli not in ('13996','16100','18400','20000','23000','02608','24000','00270','20690','20691','20692','20693','23011','99999','88888','21097')
    and pp.codvde not in ('0100','0001','0006','2319')
    AND c.codarea <> '112'
    AND c.codcid <> '0501'
GROUP BY
    pp.cod_pro,
    pp.cd_loja,
    pp.dt_emissao::date;
"""

In [4]:
dv = """
   SELECT 
    pe.cd_produto AS cod_pro,
    pe.cd_loja,
    e.dt_emissao::date AS dt_emissao,
    SUM(pe.qt_devolve) AS devolucoes
FROM "D-1".prod_ent pe
JOIN "D-1".entrada e
    ON e.cd_loja = pe.cd_loja
    AND e.sg_serie = pe.sg_serie
    AND e.nu_nota = pe.nu_nota
JOIN "D-1".cliente cli
    ON cli.codcli = pe.cd_cliente
WHERE
    e.dt_emissao >= DATE '2019-01-01'
    AND e.in_cancela = 'N'
    AND e.in_clifor = 'C'
    AND UPPER(e.nfeenvstat) NOT LIKE '%DENEG%'
    AND pe.cd_cfop NOT IN ('1949', '2949', '1603')
    AND cli.codcli NOT IN ('99999','88888','21097')
    AND cli.codcid <> '0501'
    AND cli.codarea <> '112'
    AND pe.cd_loja NOT IN ('01','08','09')
GROUP BY
    pe.cd_produto,
    pe.cd_loja,
    e.dt_emissao::date;
"""

In [5]:
grid = """
select codpro, qt_grid, cd_loja from "D-1".prd_loja 
where cd_loja not in ('08','09','01')
"""

In [6]:
with psycopg2.connect(**conn_params) as conn:
    df_grid = pd.read_sql(grid, conn)

C:\Users\priscila.marques\AppData\Local\Temp\ipykernel_14960\24163732.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_grid = pd.read_sql(grid, conn)


In [7]:
with psycopg2.connect(**conn_params) as conn:
    ven = pd.read_sql(ve, conn)

C:\Users\priscila.marques\AppData\Local\Temp\ipykernel_14960\2245771844.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ven = pd.read_sql(ve, conn)


In [8]:
with psycopg2.connect(**conn_params) as conn:
    dev = pd.read_sql(dv, conn)

C:\Users\priscila.marques\AppData\Local\Temp\ipykernel_14960\2747580985.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dev = pd.read_sql(dv, conn)


In [9]:
LEAD_TIME = 30
DATA_INICIO_SIM = "2025-01-01"
HORIZON_DIAS = 70
ALVO_LOSS = 0.05


def norm_codpro(s: pd.Series) -> pd.Series:
    return (s.astype("string")
             .str.replace(r"\.0$", "", regex=True)
             .str.strip()
             .str.zfill(6))

def norm_loja(s: pd.Series) -> pd.Series:
    return s.astype("string").str.strip()

In [10]:
ven['dt_emissao'] = pd.to_datetime(ven['dt_emissao'])
dev['dt_emissao'] = pd.to_datetime(dev['dt_emissao'])

In [11]:
ven["cod_pro"] = norm_codpro(ven["cod_pro"])
dev["cod_pro"] = norm_codpro(dev["cod_pro"])
ven["cd_loja"]  = norm_loja(ven["cd_loja"])
dev["cd_loja"]  = norm_loja(dev["cd_loja"])

df = ven.merge(dev, on=["cod_pro", "cd_loja", "dt_emissao"], how="outer")

df["vendas"] = pd.to_numeric(df["vendas"], errors="coerce").fillna(0).astype("float32")
df["devolucoes"] = pd.to_numeric(df["devolucoes"], errors="coerce").fillna(0).astype("float32")
df["demanda_dia"] = (df["vendas"] - df["devolucoes"]).clip(lower=0).astype("float32")

In [12]:
ini_2025 = pd.to_datetime("2025-01-01")
fim_2025 = pd.to_datetime("2025-12-31")

alto_giro_prod_loja = (
    df.loc[(df["dt_emissao"] >= ini_2025) & (df["dt_emissao"] <= fim_2025),
           ["cod_pro", "cd_loja", "demanda_dia"]]
      .groupby(["cod_pro", "cd_loja"], as_index=False, observed=True)
      .agg(demanda_liquida_2025=("demanda_dia", "sum"))
      .query("demanda_liquida_2025 > 108")
      [["cod_pro", "cd_loja"]]
)

print("Pares produto+loja alto giro (demanda líquida 2025 > 108):", len(alto_giro_prod_loja))

df = df.merge(alto_giro_prod_loja, on=["cod_pro", "cd_loja"], how="inner")
print("Linhas do df após filtro alto giro:", df.shape)

Pares produto+loja alto giro (demanda líquida 2025 > 108): 6287
Linhas do df após filtro alto giro: (2035456, 6)


In [13]:
dem_diaria = (
    df.groupby(["cod_pro", "cd_loja", "dt_emissao"], as_index=False, observed=True)
      .agg(demanda_dia=("demanda_dia", "sum"))
      .sort_values(["cod_pro", "cd_loja", "dt_emissao"])
)

dem_diaria = dem_diaria.set_index("dt_emissao")

dem_diaria["demanda_LT"] = (
    dem_diaria.groupby(["cod_pro", "cd_loja"], observed=True)["demanda_dia"]
              .rolling(f"{LEAD_TIME}D")
              .sum()
              .reset_index(level=[0, 1], drop=True)
)

dem_lt = dem_diaria.reset_index()

In [14]:
min_dt = dem_lt.groupby(["cod_pro", "cd_loja"], observed=True)["dt_emissao"].transform("min")
dem_lt = dem_lt[dem_lt["dt_emissao"] >= (min_dt + pd.Timedelta(days=LEAD_TIME - 1))].copy()

produtos_alvo = dem_lt[["cod_pro", "cd_loja"]].drop_duplicates()

In [15]:
df_filtrado = df.copy()
df_filtrado["dt_emissao"] = pd.to_datetime(df_filtrado["dt_emissao"]).dt.normalize()

vendas_mov = df_filtrado.loc[df_filtrado["vendas"] > 0, ["dt_emissao", "cod_pro", "cd_loja", "vendas"]].copy()
vendas_mov = vendas_mov.rename(columns={"vendas": "qtde"})
vendas_mov["qtde"] = -vendas_mov["qtde"]  # venda como saída

dev_mov = df_filtrado.loc[df_filtrado["devolucoes"] > 0, ["dt_emissao", "cod_pro", "cd_loja", "devolucoes"]].copy()
dev_mov = dev_mov.rename(columns={"devolucoes": "qtde"})  # devolução como entrada

df_movimentos = (
    pd.concat(
        [vendas_mov[["dt_emissao", "cod_pro", "cd_loja", "qtde"]],
         dev_mov[["dt_emissao", "cod_pro", "cd_loja", "qtde"]]],
        ignore_index=True
    )
    .sort_values(["cod_pro", "cd_loja", "dt_emissao"])
    .reset_index(drop=True)
)


In [16]:
df_movimentos["cod_pro"] = norm_codpro(df_movimentos["cod_pro"])
df_movimentos["cd_loja"] = norm_loja(df_movimentos["cd_loja"])


In [17]:

grid_sim = df_grid.loc[:, ["codpro", "qt_grid", "cd_loja"]].copy()
grid_sim["cod_pro"] = norm_codpro(grid_sim["codpro"])
grid_sim["cd_loja"] = norm_loja(grid_sim["cd_loja"])
grid_sim["qt_grid"] = pd.to_numeric(grid_sim["qt_grid"], errors="coerce").fillna(0).astype("int32")

grid_sim = (
    grid_sim.loc[:, ["cod_pro", "cd_loja", "qt_grid"]]
            .groupby(["cod_pro","cd_loja"], as_index=False, observed=True)
            .agg(qt_grid=("qt_grid","max"))
)

grid_map = grid_sim.set_index(["cod_pro","cd_loja"])["qt_grid"].to_dict()


In [18]:
def simular_estoque_ledger(
    movimentos_df: pd.DataFrame,
    estoque_max: int,
    lead_time_dias: int = 30,
    data_inicio_simulacao: str = "2025-01-01",
    ordenar_eventos: tuple[str, ...] = ("reabastecimento", "devolucao", "venda"),
) -> pd.DataFrame:

    dfm = movimentos_df.copy()
    dfm["dt_emissao"] = pd.to_datetime(dfm["dt_emissao"]).dt.normalize()
    inicio = pd.to_datetime(data_inicio_simulacao).normalize()
    dfm = dfm[dfm["dt_emissao"] >= inicio].sort_values("dt_emissao").reset_index(drop=True)

    if dfm.empty:
        return pd.DataFrame(columns=[
            "dt_emissao", "movimento", "qtde",
            "estoque_inicial", "estoque_final",
            "venda_perdida", "origem_sale_dt",
            "cod_pro", "cd_loja"
        ])

    dfm["dia"] = dfm["dt_emissao"]

    cod_pro = dfm["cod_pro"].iloc[0] if "cod_pro" in dfm.columns else None
    cd_loja = dfm["cd_loja"].iloc[0] if "cd_loja" in dfm.columns else None

    pedidos = deque()  # FIFO de pedidos
    estoque = int(estoque_max)  # começa cheio
    historico = []

    dia_min = dfm["dia"].min()
    dia_max = dfm["dia"].max() + pd.Timedelta(days=lead_time_dias)
    dias_unicos = pd.date_range(dia_min, dia_max, freq="D")

    ordem = {nome: i for i, nome in enumerate(ordenar_eventos)}
    mov_por_dia = {d: g for d, g in dfm.groupby("dia", sort=False)}

    for dia in dias_unicos:
        # 1) aplica reposições vencidas até o dia
        while pedidos and pedidos[0]["arrival_dt"] <= dia:
            ped = pedidos[0]
            if estoque >= estoque_max:
                break

            cabe = estoque_max - estoque
            entra = min(int(ped["qty_remaining"]), int(cabe))
            if entra <= 0:
                break

            estoque_inicial = estoque
            estoque += int(entra)
            ped["qty_remaining"] -= int(entra)

            historico.append({
                "dt_emissao": dia,
                "movimento": "reabastecimento",
                "qtde": float(entra),
                "estoque_inicial": int(estoque_inicial),
                "estoque_final": int(estoque),
                "venda_perdida": 0.0,
                "origem_sale_dt": ped["sale_dt"],
                "cod_pro": cod_pro,
                "cd_loja": cd_loja,
            })

            if ped["qty_remaining"] <= 0:
                pedidos.popleft()

        # 2) movimentos do dia (processa TODAS as linhas)
        mov_dia = mov_por_dia.get(dia)
        if mov_dia is not None:
            for _, row in mov_dia.iterrows():
                qtde = float(row["qtde"])

                if qtde > 0:  # devolução
                    estoque_inicial = estoque
                    entra = min(int(qtde), int(estoque_max - estoque))
                    estoque += int(entra)

                    historico.append({
                        "dt_emissao": dia,
                        "movimento": "devolucao",
                        "qtde": float(qtde),
                        "estoque_inicial": int(estoque_inicial),
                        "estoque_final": int(estoque),
                        "venda_perdida": 0.0,
                        "origem_sale_dt": None,
                        "cod_pro": cod_pro,
                        "cd_loja": cd_loja,
                    })

                else:  # venda
                    demanda = int(round(-qtde))
                    estoque_inicial = estoque

                    if estoque >= demanda:
                        vendido = demanda
                        perdido = 0
                        estoque -= int(vendido)
                    else:
                        vendido = int(estoque)
                        perdido = int(demanda - estoque)
                        estoque = 0

                    historico.append({
                        "dt_emissao": dia,
                        "movimento": "venda",
                        "qtde": float(qtde),
                        "estoque_inicial": int(estoque_inicial),
                        "estoque_final": int(estoque),
                        "venda_perdida": float(perdido),
                        "origem_sale_dt": None,
                        "cod_pro": cod_pro,
                        "cd_loja": cd_loja,
                    })

                    # gera pedido só do que realmente vendeu
                    if vendido > 0:
                        arrival = dia + pd.Timedelta(days=lead_time_dias)
                        pedidos.append({
                            "arrival_dt": arrival,
                            "qty_remaining": int(vendido),
                            "sale_dt": dia,
                        })

        # 3) se devolução abriu espaço, tenta entrar mais reposição vencida hoje
        while pedidos and pedidos[0]["arrival_dt"] <= dia and estoque < estoque_max:
            ped = pedidos[0]
            cabe = estoque_max - estoque
            entra = min(int(ped["qty_remaining"]), int(cabe))
            if entra <= 0:
                break

            estoque_inicial = estoque
            estoque += int(entra)
            ped["qty_remaining"] -= int(entra)

            historico.append({
                "dt_emissao": dia,
                "movimento": "reabastecimento",
                "qtde": float(entra),
                "estoque_inicial": int(estoque_inicial),
                "estoque_final": int(estoque),
                "venda_perdida": 0.0,
                "origem_sale_dt": ped["sale_dt"],
                "cod_pro": cod_pro,
                "cd_loja": cd_loja,
            })

            if ped["qty_remaining"] <= 0:
                pedidos.popleft()

    out = pd.DataFrame(historico)
    out["mov_ord"] = out["movimento"].map(ordem).fillna(999).astype(int)
    out = out.sort_values(["dt_emissao", "mov_ord"]).drop(columns=["mov_ord"]).reset_index(drop=True)
    return out

In [19]:
def loss_rate_70d_ledger(grupo_mov: pd.DataFrame, S: int,
                         lead_time: int = 30,
                         inicio: str = "2025-01-01",
                         horizonte_dias: int = 70) -> float:
    g = grupo_mov.copy()
    g["dt_emissao"] = pd.to_datetime(g["dt_emissao"]).dt.normalize()

    dt_ini = pd.to_datetime(inicio).normalize()
    dt_fim = dt_ini + pd.Timedelta(days=horizonte_dias)

    g = g[(g["dt_emissao"] >= dt_ini) & (g["dt_emissao"] < dt_fim)].sort_values("dt_emissao")
    if g.empty:
        return 0.0

    sim = simular_estoque_ledger(
        movimentos_df=g[["dt_emissao","cod_pro","cd_loja","qtde"]],
        estoque_max=int(S),
        lead_time_dias=lead_time,
        data_inicio_simulacao=inicio
    )

    demanda_total = float((-g.loc[g["qtde"] < 0, "qtde"]).sum())
    perdida = float(sim["venda_perdida"].sum())

    return 0.0 if demanda_total <= 0 else float(perdida / demanda_total)


In [20]:
def refinar_S_minimo_partindo_da_grid(grupo_mov: pd.DataFrame,
                                      S_start: int,
                                      alvo_loss: float = 0.05,
                                      lead_time: int = 30,
                                      inicio: str = "2025-01-01",
                                      horizonte_dias: int = 70) -> dict:
    cache: dict[int, float] = {}

    def lr(S: int) -> float:
        S = int(max(0, S))
        if S not in cache:
            cache[S] = float(loss_rate_70d_ledger(grupo_mov, S, lead_time, inicio, horizonte_dias))
        return cache[S]

    S = int(max(0, S_start))
    loss_S = lr(S)

    # se por algum motivo S_start não está ok, sobe até ficar ok
    while loss_S > alvo_loss:
        S += 1
        loss_S = lr(S)

    # agora desce até quebrar
    while S > 0:
        loss_prev = lr(S - 1)
        if loss_prev <= alvo_loss:
            S -= 1
            loss_S = loss_prev
        else:
            return {
                "S_ok": int(S),
                "loss_ok": float(loss_S),
                "S_quebra": int(S - 1),
                "loss_quebra": float(loss_prev),
            }

    return {"S_ok": int(S), "loss_ok": float(loss_S), "S_quebra": None, "loss_quebra": None}


In [21]:
#quando ja tiver os estoques e nao quer esperar muito tempo (demora cerca de 5min):
#df_S_otimo = pd.read_parquet("df_S_otimo.parquet")

In [ ]:
resultados_refino_grid = []

grupos = df_movimentos.groupby(["cod_pro", "cd_loja"], sort=False)
total = grupos.ngroups

for i, ((prod, loja), grupo) in enumerate(grupos, start=1):
    S_grid = int(grid_map.get((prod, loja), 0))
    if S_grid <= 0:
        continue

    r = refinar_S_minimo_partindo_da_grid(
        grupo_mov=grupo,
        S_start=S_grid,
        alvo_loss=ALVO_LOSS,
        lead_time=LEAD_TIME,
        inicio=DATA_INICIO_SIM,
        horizonte_dias=HORIZON_DIAS
    )

    resultados_refino_grid.append({
        "cod_pro": str(prod),
        "cd_loja": str(loja),
        "S_grid": int(S_grid),
        "S_refinado_grid": int(r["S_ok"]),
        "loss_refinado": float(r["loss_ok"]),
        "S_quebra": r["S_quebra"],
        "loss_quebra": r["loss_quebra"],
        "economia_unidades": int(S_grid - r["S_ok"]),
    })

    if i % 200 == 0:
        print(f"{i}/{total} refinados a partir da GRID...")

df_S_refinado = pd.DataFrame(resultados_refino_grid)

out_path = os.path.join(os.getcwd(), "df_S_refinado_grid.parquet")
df_S_refinado.to_parquet(out_path, index=False)
print("Salvo em:", out_path)

200/6287 refinados a partir da GRID...
400/6287 refinados a partir da GRID...
600/6287 refinados a partir da GRID...
800/6287 refinados a partir da GRID...
1000/6287 refinados a partir da GRID...
1200/6287 refinados a partir da GRID...
1400/6287 refinados a partir da GRID...
1600/6287 refinados a partir da GRID...
1800/6287 refinados a partir da GRID...
2000/6287 refinados a partir da GRID...
2200/6287 refinados a partir da GRID...
2400/6287 refinados a partir da GRID...
2600/6287 refinados a partir da GRID...
2800/6287 refinados a partir da GRID...
3000/6287 refinados a partir da GRID...
3200/6287 refinados a partir da GRID...
3400/6287 refinados a partir da GRID...
3600/6287 refinados a partir da GRID...
3800/6287 refinados a partir da GRID...
4000/6287 refinados a partir da GRID...
4200/6287 refinados a partir da GRID...
4400/6287 refinados a partir da GRID...
4600/6287 refinados a partir da GRID...


In [ ]:
def auditar_S(prod: str, loja: str, modo: str = "grid"):
    prod_n = norm_codpro(pd.Series([str(prod)])).iloc[0]
    loja_n = norm_loja(pd.Series([str(loja)])).iloc[0]
    modo = str(modo).strip().lower()

    # grid
    S_grid = int(grid_map.get((prod_n, loja_n), 0))

    # refinado
    rowR = df_S_refinado[(df_S_refinado["cod_pro"] == prod_n) & (df_S_refinado["cd_loja"] == loja_n)]
    S_ref = int(rowR["S_refinado_grid"].iloc[0]) if not rowR.empty else None

    if modo in ("grid", "g"):
        S_usado = S_grid
        label = "GRID"
    elif modo in ("ref", "r", "refinado"):
        if S_ref is None:
            print("Não achei refinado para esse produto/loja.")
            return pd.DataFrame()
        S_usado = S_ref
        label = "S_REFINADO_GRID_5%"
    else:
        raise ValueError("modo inválido. Use 'grid' ou 'ref'.")

    # movimentos do par no horizonte
    g = df_movimentos[(df_movimentos["cod_pro"] == prod_n) & (df_movimentos["cd_loja"] == loja_n)].copy()
    g["dt_emissao"] = pd.to_datetime(g["dt_emissao"]).dt.normalize()

    dt_ini = pd.to_datetime(DATA_INICIO_SIM).normalize()
    dt_fim = dt_ini + pd.Timedelta(days=HORIZON_DIAS)

    g = g[(g["dt_emissao"] >= dt_ini) & (g["dt_emissao"] < dt_fim)].sort_values("dt_emissao")
    if g.empty:
        print(f"Sem movimentos no horizonte de {HORIZON_DIAS} dias")
        return pd.DataFrame()

    sim = simular_estoque_ledger(
        movimentos_df=g[["dt_emissao", "cod_pro", "cd_loja", "qtde"]],
        estoque_max=int(S_usado),
        lead_time_dias=LEAD_TIME,
        data_inicio_simulacao=DATA_INICIO_SIM
    )

    demanda_total = float((-g.loc[g["qtde"] < 0, "qtde"]).sum())
    perdida = float(sim["venda_perdida"].sum())

    media_dia = demanda_total / HORIZON_DIAS if HORIZON_DIAS > 0 else 0.0
    dias_cobertura = (S_usado / media_dia) if media_dia > 0 else np.nan

    print(f"Produto {prod_n} | Loja {loja_n} | Modo: {label}")
    print(f"S usado: {S_usado}  (grid={S_grid} | refinado={S_ref if S_ref is not None else 'NA'})")
    print(f"Horizonte: {HORIZON_DIAS} dias a partir de {DATA_INICIO_SIM}")
    print(f"Demanda total (70d): {demanda_total:.0f}")
    print(f"Venda perdida (70d): {perdida:.0f}")
    print(f"Média/dia (70d): {media_dia:.2f} | Cobertura do S: {dias_cobertura:.1f} dias")

    cols_final = [
        "cod_pro","cd_loja","dt_emissao","estoque_inicial","qtde",
        "estoque_final","venda_perdida","movimento","origem_sale_dt"
    ]
    return sim.loc[:, cols_final]

In [ ]:
auditar_S("062470", "04", "grid") 
auditar_S("062470", "04", "ref")  

Produto 062470 | Loja 04 | Modo: GRID
S usado: 16  (grid=16 | refinado=39)
Horizonte: 70 dias a partir de 2025-01-01
Demanda total (70d): 24
Venda perdida (70d): 0
Média/dia (70d): 0.34 | Cobertura do S: 46.7 dias
Produto 062470 | Loja 04 | Modo: S_REFINADO_5%
S usado: 39  (grid=16 | refinado=39)
Horizonte: 70 dias a partir de 2025-01-01
Demanda total (70d): 24
Venda perdida (70d): 0
Média/dia (70d): 0.34 | Cobertura do S: 113.8 dias


,cod_pro,cd_loja,dt_emissao,estoque_inicial,qtde,estoque_final,venda_perdida,movimento,origem_sale_dt
0,062470,04,2025-01-06,39,-8.0,31,0.0,venda,NaT
1,062470,04,2025-01-08,31,-8.0,23,0.0,venda,NaT
2,062470,04,2025-01-09,23,8.0,31,0.0,devolucao,NaT
3,062470,04,2025-02-05,31,8.0,39,0.0,reabastecimento,2025-01-06
4,062470,04,2025-02-10,31,8.0,39,0.0,reabastecimento,2025-01-08
5,062470,04,2025-02-10,39,-8.0,31,0.0,venda,NaT


In [ ]:
auditar_S("019561", "06", "grid") 

Produto 019561 | Loja 06 | Modo: GRID
S usado: 6  (grid=6 | refinado=18)
Horizonte: 70 dias a partir de 2025-01-01
Demanda total (70d): 22
Venda perdida (70d): 13
Média/dia (70d): 0.31 | Cobertura do S: 19.1 dias


,cod_pro,cd_loja,dt_emissao,estoque_inicial,qtde,estoque_final,venda_perdida,movimento,origem_sale_dt
0,019561,06,2025-01-03,6,-1.0,5,0.0,venda,NaT
1,019561,06,2025-01-06,5,-1.0,4,0.0,venda,NaT
2,019561,06,2025-01-13,4,-8.0,0,4.0,venda,NaT
3,019561,06,2025-01-21,0,-4.0,0,4.0,venda,NaT
4,019561,06,2025-02-02,0,1.0,1,0.0,reabastecimento,2025-01-03
5,019561,06,2025-02-03,1,-2.0,0,1.0,venda,NaT
6,019561,06,2025-02-04,0,-4.0,0,4.0,venda,NaT
7,019561,06,2025-02-05,0,1.0,1,0.0,reabastecimento,2025-01-06
8,019561,06,2025-02-12,1,4.0,5,0.0,reabastecimento,2025-01-13
9,019561,06,2025-02-13,5,-1.0,4,0.0,venda,NaT


In [ ]:
auditar_S("019561", "06", "ref")  

Sem movimentos no horizonte de 70 dias


""


In [ ]:
#df_comp_servico.to_excel("comp_servico.xlsx",index=False)